<a href="https://colab.research.google.com/github/usma11dia0/web_scraping_on_colab/blob/main/web_scraping_on_colab_yahoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q google-colab-selenium

In [ ]:
#環境変数設定
TARGET_URL = 'https://finance.yahoo.co.jp/cm/message/1003778/3778/155'
DRIVER_WAIT_TIME = 2
SCROLL_PAUSE_TIME = 1
MAX_SCROLL_ATTEMPTS = 5

# 各種ファイル保存先
SUBMIT_FILE_PATH ='/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/comments_list_yahoo.xlsx'

#取得データ格納先
result_dict = {
    'ID': [],
    'コメント':[],
    '返信元ID': []
}

In [ ]:
#標準ライブラリ
import os
import math
import json
import re
import time


#サードパーティライブラリ
import pandas as pd
import google_colab_selenium as gs
from google.colab import drive
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from logging import (
    getLogger,
    StreamHandler,
    DEBUG,
    INFO,
    Formatter,
    config
)
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#関数定義
#スクロールしてページを更新
def scroll_page(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    attempts = 0
    while attempts < MAX_SCROLL_ATTEMPTS:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        attempts += 1

In [ ]:
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--disable-extensions")
options.add_argument("--disable-popup-blocking")
options.add_argument(
       "user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"
    )
options.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2,  # 画像の無効化
})

driver = gs.Chrome(options=options)
driver.implicitly_wait(DRIVER_WAIT_TIME)

<IPython.core.display.Javascript object>

In [ ]:
#カスタムロガーの設定
with open('/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/logging_config.json', 'r') as f:
    logger_config = json.load(f)
config.dictConfig(logger_config)
logger = getLogger('main')

try:
      driver.get(TARGET_URL)
      logger.debug(f'{TARGET_URL}：トップページへ移動しました')
except Exception as e:
      logger.error(f'{TARGET_URL}：トップページへ移動出来ませんでした: [e]')
      raise

2024-02-14 20:43:35,909 [DEBUG] main: https://finance.yahoo.co.jp/cm/message/1003778/3778/155：トップページへ移動しました


In [ ]:
#コメント抽出
ult_element = driver.find_element(By.ID,"cmtlst")
comment_list_element = ult_element.find_element(By.CSS_SELECTOR,".commentList")
data_comment_elements = comment_list_element.find_elements(By.CSS_SELECTOR, 'li[data-comment]')

for data_comment_element in data_comment_elements:
    # ID
    id = data_comment_element.find_element(By.CSS_SELECTOR, ".comNum").text
    # コメント
    comment = data_comment_element.find_element(By.CSS_SELECTOR, ".comText").text
    #返答先ID
    reply_to_elements = data_comment_element.find_elements(By.CSS_SELECTOR, ".comReplyTo")
    reply_to_text = reply_to_elements[0].text if reply_to_elements else '-'

    #結果格納
    result_dict['ID'].append(id)
    result_dict['コメント'].append(comment)
    result_dict['返信元ID'].append(reply_to_text)

print(result_dict)

{'ID': ['2421（最新）', '2420', '2419', '2418', '2417', '2416', '2413', '2412', '2411', '2410', '2409', '2407', '2406', '2405', '2404', '2403', '2402', '2401', '2399', '2398', '2421（最新）', '2420', '2419', '2418', '2417', '2416', '2413', '2412', '2411', '2410', '2409', '2407', '2406', '2405', '2404', '2403', '2402', '2401', '2399', '2398'], 'コメント': ['分割してくれてもええんやで（切実）', '>ここは700円くらいが適正なのは事実だろ？？\n>明日はストップ安だよ\n\n最高値ということは、全員が含み益な状態です。\n\nつまり、あなたが売りポジションということがわかります。買いと違って売りは青天井です。お気をつけください。', 'PTS4,900円', 'A Iの技術が生かされるのは5年〜10年後それを踏まえて新ニーサで買って人気銘柄\u3000人気が一番高かった購入時の株価は4000〜4100円まだ1ヶ月も経っていない\u3000現在の4800円の株価で手放す個人投資家は少ないと思う', 'なんかAIですらないbotみたいな奴いるなあｗ', '売ってしまったやつら、ざまー', '( ´∀｀)激しく同意', '焦り過ぎで見透かされ！', '億り人確定の皆さま、こんにちは', 'うわばなれ\u3000なれ', 'ピーティーエスは騙し上げだ\nこんな少ない出来高を信用する〇〇いるなんて楽観すぎるよ。明日はストップ安', '( ＾∀＾)ダメダコリャ…\nこの時間のP上昇、エヌも上昇。この意味がわからんのかねぇ\n上げに乗れなかったのかなぁ', '明日はストップ安\n明後日もストップ安', '4連ストップ安が見れるぞ', 'ここは700円くらいが適正なのは事実だろ？？\n明日はストップ安だよ', '( ＾∀＾)カワイソウニ…', '値下がりランキングしばらく顔出さない→ストップ高→値崩れ調整\nがどうしても脳裏を横切りますが、と見せ掛けて

In [ ]:
import pandas as pd
df = pd.DataFrame(result_dict)
df

2024-02-14 20:41:52,422 [INFO] llmx: Info: LLMX_CONFIG_PATH environment variable is not set to a valid config file. Using default config file at '/usr/local/lib/python3.10/dist-packages/llmx/configs/config.default.yml'.
2024-02-14 20:41:52,444 [INFO] llmx: Loaded config from '/usr/local/lib/python3.10/dist-packages/llmx/configs/config.default.yml'.


,ID,コメント,返信元ID
0,2421（最新）,分割してくれてもええんやで（切実）,-
1,2420,>ここは700円くらいが適正なのは事実だろ？？\n>明日はストップ安だよ\n\n最高値という...,>>2404
2,2419,"PTS4,900円",-
3,2418,A Iの技術が生かされるのは5年〜10年後それを踏まえて新ニーサで買って人気銘柄 人気が一番...,-
4,2417,なんかAIですらないbotみたいな奴いるなあｗ,-
5,2416,売ってしまったやつら、ざまー,-
6,2413,( ´∀｀)激しく同意,>>2412
7,2412,焦り過ぎで見透かされ！,>>2406
8,2411,億り人確定の皆さま、こんにちは,-
9,2410,うわばなれ なれ,-


In [ ]:
# エクセルファイルとして保存
df.to_excel(SUBMIT_FILE_PATH, header=True, index=False)

# 新しいセクション